# Analyse des Spectrogrammes NightScan

Ce notebook permet d'analyser et visualiser les spectrogrammes générés avec différentes configurations.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchaudio
import torchaudio.transforms as T
from pathlib import Path
import pandas as pd
import seaborn as sns
from IPython.display import Audio, display

# Importer nos modules
import sys
sys.path.append('..')
from spectrogram_config import SpectrogramConfig, get_config_for_animal, ANIMAL_FREQ_RANGES
from audio_augmentation import AudioAugmentation, PreprocessingPipeline
from audio_dataset import AudioSpectrogramDataset

# Configuration du style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

## 1. Comparaison des configurations par type d'animal

In [ ]:
# Afficher les paramètres pour chaque type d'animal
config_comparison = []

for animal_type in ANIMAL_FREQ_RANGES.keys():
    config = get_config_for_animal(animal_type)
    config_comparison.append({
        'Type': animal_type,
        'Sample Rate': config.sample_rate,
        'n_mels': config.n_mels,
        'fmin': config.fmin,
        'fmax': config.fmax,
        'n_fft': config.n_fft,
        'hop_length': config.hop_length
    })

df_config = pd.DataFrame(config_comparison)
display(df_config)

## 2. Visualisation des plages de fréquences

In [ ]:
# Visualiser les plages de fréquences pour chaque animal
fig, ax = plt.subplots(figsize=(12, 6))

y_pos = np.arange(len(ANIMAL_FREQ_RANGES))
animals = list(ANIMAL_FREQ_RANGES.keys())

for i, animal in enumerate(animals):
    config = get_config_for_animal(animal)
    ax.barh(i, config.fmax - config.fmin, left=config.fmin, height=0.6,
            label=f"{animal} ({config.fmin}-{config.fmax} Hz)")

ax.set_yticks(y_pos)
ax.set_yticklabels(animals)
ax.set_xlabel('Fréquence (Hz)')
ax.set_title('Plages de fréquences par type d\'animal')
ax.set_xscale('log')
ax.grid(True, axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

## 3. Génération et comparaison de spectrogrammes

In [ ]:
def generate_test_signal(duration=3, sample_rate=22050, freq_components=None):
    """Génère un signal de test avec plusieurs composantes fréquentielles."""
    t = torch.linspace(0, duration, int(sample_rate * duration))
    
    if freq_components is None:
        freq_components = [(1000, 0.5), (2000, 0.3), (4000, 0.2)]
    
    signal = torch.zeros_like(t)
    for freq, amp in freq_components:
        signal += amp * torch.sin(2 * np.pi * freq * t)
    
    # Ajouter un peu de bruit
    signal += 0.05 * torch.randn_like(signal)
    
    return signal.unsqueeze(0)  # Ajouter dimension channel

def plot_spectrogram(spec, title, ax, sample_rate=22050, hop_length=512):
    """Affiche un spectrogramme."""
    img = ax.imshow(spec, aspect='auto', origin='lower', cmap='viridis')
    
    # Configurer les axes
    ax.set_title(title)
    ax.set_xlabel('Temps (s)')
    ax.set_ylabel('Fréquence (mel)')
    
    # Ajuster les labels de temps
    time_frames = spec.shape[1]
    time_seconds = time_frames * hop_length / sample_rate
    ax.set_xlim(0, time_frames)
    
    # Créer des ticks de temps
    n_ticks = 5
    tick_locs = np.linspace(0, time_frames, n_ticks)
    tick_labels = np.linspace(0, time_seconds, n_ticks)
    ax.set_xticks(tick_locs)
    ax.set_xticklabels([f'{t:.1f}' for t in tick_labels])
    
    return img

In [ ]:
# Comparer les spectrogrammes pour différentes configurations
test_signal = generate_test_signal(duration=3)

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, animal_type in enumerate(['general', 'bird_song', 'owl', 'bat', 'insect', 'amphibian']):
    config = get_config_for_animal(animal_type)
    
    # Adapter le signal au sample rate de la config
    if config.sample_rate != 22050:
        resampler = T.Resample(22050, config.sample_rate)
        signal = resampler(test_signal)
    else:
        signal = test_signal
    
    # Créer le spectrogramme
    mel_transform = T.MelSpectrogram(
        sample_rate=config.sample_rate,
        n_mels=config.n_mels,
        n_fft=config.n_fft,
        hop_length=config.hop_length,
        f_min=config.fmin,
        f_max=config.fmax
    )
    
    mel_spec = mel_transform(signal)
    mel_spec_db = T.AmplitudeToDB(top_db=config.top_db)(mel_spec)
    
    plot_spectrogram(
        mel_spec_db.squeeze().numpy(),
        f'{animal_type}\n(sr={config.sample_rate}, fmin={config.fmin}, fmax={config.fmax})',
        axes[i],
        config.sample_rate,
        config.hop_length
    )

plt.tight_layout()
plt.show()

## 4. Analyse de l'augmentation des données

In [ ]:
# Créer un signal de test
original_signal = generate_test_signal(duration=3)
augmenter = AudioAugmentation()

# Appliquer différentes augmentations
augmentations = {
    'Original': original_signal,
    'Avec bruit': augmenter.add_noise(original_signal, noise_level=0.1),
    'Décalage temporel': augmenter.time_shift(original_signal),
    'Changement vitesse': augmenter.change_speed(original_signal, speed_factor=1.1),
    'Pitch shift': augmenter.pitch_shift(original_signal, n_steps=2)
}

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

config = SpectrogramConfig()  # Config par défaut
mel_transform = T.MelSpectrogram(
    sample_rate=config.sample_rate,
    n_mels=config.n_mels,
    n_fft=config.n_fft,
    hop_length=config.hop_length
)

for i, (name, signal) in enumerate(augmentations.items()):
    if i >= len(axes):
        break
        
    mel_spec = mel_transform(signal)
    mel_spec_db = T.AmplitudeToDB(top_db=80)(mel_spec)
    
    plot_spectrogram(
        mel_spec_db.squeeze().numpy(),
        name,
        axes[i],
        config.sample_rate,
        config.hop_length
    )

# Cacher le dernier subplot vide
if len(augmentations) < len(axes):
    axes[-1].set_visible(False)

plt.tight_layout()
plt.show()

## 5. Analyse SpecAugment

In [ ]:
# Démonstration de SpecAugment
original_spec = mel_transform(original_signal)
original_spec_db = T.AmplitudeToDB(top_db=80)(original_spec)

# Appliquer SpecAugment plusieurs fois
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

plot_spectrogram(
    original_spec_db.squeeze().numpy(),
    'Spectrogramme original',
    axes[0],
    config.sample_rate,
    config.hop_length
)

for i in range(1, 4):
    augmented_spec = augmenter.spec_augment(original_spec_db.clone())
    plot_spectrogram(
        augmented_spec.squeeze().numpy(),
        f'SpecAugment #{i}',
        axes[i],
        config.sample_rate,
        config.hop_length
    )

plt.tight_layout()
plt.show()

## 6. Analyse des vrais données (si disponibles)

In [ ]:
# Charger et analyser de vraies données si disponibles
csv_path = Path('../data/processed/csv/train.csv')
audio_dir = Path('../audio_data')

if csv_path.exists() and audio_dir.exists():
    # Charger le dataset
    dataset = AudioSpectrogramDataset(
        csv_file=csv_path,
        audio_dir=audio_dir,
        animal_type='general',
        augment=False
    )
    
    if len(dataset) > 0:
        print(f"Dataset chargé: {len(dataset)} échantillons")
        print(f"Classes: {dataset.class_names}")
        
        # Visualiser quelques échantillons par classe
        n_samples_per_class = 2
        n_classes = len(dataset.class_names)
        
        fig, axes = plt.subplots(n_classes, n_samples_per_class, 
                                figsize=(n_samples_per_class * 5, n_classes * 3))
        
        if n_classes == 1:
            axes = axes.reshape(1, -1)
        
        # Grouper par classe
        samples_by_class = {class_name: [] for class_name in dataset.class_names}
        
        for i in range(len(dataset)):
            spec, label_idx = dataset[i]
            class_name = dataset.class_names[label_idx]
            if len(samples_by_class[class_name]) < n_samples_per_class:
                samples_by_class[class_name].append((spec, i))
        
        # Afficher
        for class_idx, class_name in enumerate(dataset.class_names):
            for sample_idx, (spec, dataset_idx) in enumerate(samples_by_class[class_name][:n_samples_per_class]):
                ax = axes[class_idx, sample_idx] if n_classes > 1 else axes[sample_idx]
                
                # Prendre le premier canal si RGB
                if spec.shape[0] == 3:
                    spec_to_plot = spec[0].numpy()
                else:
                    spec_to_plot = spec.numpy()
                
                img = ax.imshow(spec_to_plot, aspect='auto', origin='lower', cmap='viridis')
                ax.set_title(f'{class_name} - Sample {dataset_idx}')
                ax.set_xlabel('Temps')
                ax.set_ylabel('Fréquence (mel)')
        
        plt.tight_layout()
        plt.show()
        
        # Statistiques sur les spectrogrammes
        print("\nStatistiques des spectrogrammes:")
        specs = []
        for i in range(min(100, len(dataset))):
            spec, _ = dataset[i]
            specs.append(spec.numpy())
        
        specs = np.array(specs)
        print(f"Forme: {specs.shape}")
        print(f"Min: {specs.min():.2f}, Max: {specs.max():.2f}")
        print(f"Moyenne: {specs.mean():.2f}, Écart-type: {specs.std():.2f}")
        
else:
    print("Pas de données réelles trouvées. Exécutez d'abord prepare_audio_data.py")

## 7. Recommandations basées sur l'analyse

### Paramètres optimaux détectés:

1. **Général (défaut)**:
   - Sample rate: 22050 Hz
   - n_mels: 128
   - fmin: 50 Hz, fmax: 11000 Hz
   - Bon compromis pour la plupart des animaux

2. **Chauves-souris**:
   - Sample rate: 192000 Hz (pour capturer les ultrasons)
   - n_mels: 256 (plus de résolution)
   - fmin: 15000 Hz, fmax: 100000 Hz

3. **Chouettes**:
   - Sample rate: 16000 Hz (suffisant pour basses fréquences)
   - fmin: 200 Hz, fmax: 4000 Hz

### Augmentation des données:

- **SpecAugment** très efficace pour la généralisation
- **Bruit gaussien** simule les conditions réelles
- **Décalage temporel** pour la robustesse
- **Changement de vitesse** simule différentes distances

### Prétraitement recommandé:

1. Filtre passe-haut (50 Hz) pour éliminer le bruit
2. Normalisation par spectrogramme
3. Conversion en RGB pour compatibilité avec les modèles pré-entraînés

In [ ]:
# Sauvegarder un rapport
report = {
    'date': pd.Timestamp.now().isoformat(),
    'configurations': df_config.to_dict(),
    'recommendations': {
        'general': 'Utiliser la configuration par défaut pour commencer',
        'bat_detection': 'Nécessite matériel spécialisé (microphone ultrasonique)',
        'augmentation': 'Activer pour améliorer la généralisation',
        'preprocessing': 'Toujours filtrer < 50 Hz'
    }
}

import json
with open('spectrogram_analysis_report.json', 'w') as f:
    json.dump(report, f, indent=2)

print("Rapport sauvegardé dans spectrogram_analysis_report.json")